In [1]:
import os

In [2]:
os.getcwd()

'/home/arsalan/Public/uni/sem3/dbs/project/AgroNew/data/notebooks'

In [17]:
import numpy as np

In [4]:
os.path.exists(file_path)

True

In [5]:
%pip install h5py

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import h5py

In [19]:
with h5py.File(file_path, 'r+') as file:
    # List all groups and datasets in the file
    print("Groups and Datasets: ")
    print(list(file.keys()))

OSError: Unable to synchronously open file (file signature not found)

In [20]:
%pip install pyhdf

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.4/771.4 KB 424.7 kB/s eta 0:00:001m439.6 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyhdf import SD

In [6]:
file_path = "../ndvi/MOD13A1.A2023321.{h23v05}.061.2023340004705.hdf"

'../ndvi/MOD13A1.A2023321.h23v5.061.2023340004705.hdf'

In [7]:
hdf_file = SD.SD(file_path.format("h23v05"), SD.SDC.READ)

In [8]:
print("Datasets in the HDF-EOS file:")
for dataset_name in hdf_file.datasets().keys():
    print(dataset_name)

Datasets in the HDF-EOS file:
500m 16 days NDVI
500m 16 days EVI
500m 16 days VI Quality
500m 16 days red reflectance
500m 16 days NIR reflectance
500m 16 days blue reflectance
500m 16 days MIR reflectance
500m 16 days view zenith angle
500m 16 days sun zenith angle
500m 16 days relative azimuth angle
500m 16 days composite day of the year
500m 16 days pixel reliability


In [10]:
dataset_ndvi = hdf_file.datasets()["500m 16 days NDVI"]

In [11]:
dataset = hdf_file.select("500m 16 days NDVI")

In [13]:
dataset.attributes()

{'long_name': '500m 16 days NDVI',
 'units': 'NDVI',
 'valid_range': [-2000, 10000],
 '_FillValue': -3000,
 'scale_factor': 10000.0,
 'scale_factor_err': 0.0,
 'add_offset': 0.0,
 'add_offset_err': 0.0,
 'calibrated_nt': 5}

In [14]:
dataset.get().shape

(2400, 2400)

In [15]:
ndvi_h23v05_data = dataset.get() 

In [16]:
ndvi_h23v05_data

array([[ 994,  920,  920, ...,  774,  745,  661],
       [ 877,  994,  930, ...,  675,  552,  483],
       [ 863,  947,  932, ...,  490,  559,  477],
       ...,
       [1086, 1089, 1097, ..., 2206, 2263, 2253],
       [1104, 1069,  911, ..., 2272, 2392, 2584],
       [1065,  646,  657, ..., 2553, 2486, 2559]], dtype=int16)

In [20]:
## Always remember the resolution of each granule
# For now NDVI is 500m while LST is 1km
# Hence NDVI resolution is 0.5 while LST is 1

TILEVH = {
    'h23v05': [(57.7350,    78.3353),   (30.0000,   40.0000)],
    'h23v06': [(53.2089,    69.2917),   (20.0000,   30.0000)],
    'h24v05': [(69.2820,    91.3894),   (30.0000,   40.0000)],
    'h24v06': [(63.8507,    80.8387),   (20.0000,   30.0000)],
}

In [23]:
TILEVH['h23v05'][0][0]

57.735

In [34]:
from math import floor
def return_coordinates(long, lat, resolution):
    
    total_points = 1200/resolution

    tile = ""

    for each in TILEVH.keys():
        if long > TILEVH[each][0][0] and long < TILEVH[each][0][1] and lat > TILEVH[each][1][0] and lat < TILEVH[each][1][1]:
            tile = each
            break

    if tile == "":
        raise ValueError("Long and Latitudes are incorrect please provide again in order (long, lat)")

    rel_long = long - TILEVH[tile][0][0]
    print("rel_long", rel_long)
    rel_lat = lat - TILEVH[tile][1][0]
    long_dist = TILEVH[tile][0][1] - TILEVH[tile][0][0]
    lat_dist = TILEVH[tile][1][1] - TILEVH[tile][1][0]

    x = floor((rel_long/long_dist) * total_points)
    y = floor((rel_lat/lat_dist) * total_points)

    return (x, y, tile)

In [37]:
return_coordinates(73, 33, 1)

rel_long 15.265


(889, 360, 'h23v05')